In [ ]:
# 必要なライブラリのインストール
# !pip install google-api-python-client

In [ ]:
import pandas as pd
from googleapiclient.discovery import build

In [ ]:
# Google の生成した API キー
API_KEY = '*********************'

# Aoi ch. の チャンネルID
ch_id = 'UC3Ruo_5doyu514PesWGvCAg'

# part
part='snippet'

# API
https://googleapis.github.io/google-api-python-client/docs/dyn/youtube_v3.html

In [ ]:
# youtube オブジェクトを作成する
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [ ]:
def get_video_info(part, ch_id, order='viewCount', max_results='50'):
    """
    チャンネルに属する動画を全量取得します。
    
    part: 
    ch_id: チャンネルの固有ID
    order: 動画を取得する際のソート方法
    max_result: 1回の問い合わせで取得する動画の本数。上限は "50" です。
    """
    dic_list = []
    search_response = youtube.search().list(part=part,channelId=ch_id, order=order, maxResults=max_results)
    output = youtube.search().list(part=part,channelId=ch_id, order=order, maxResults=max_results).execute()

    # データを取得しきるまでループします。
    # あまりにも量が多い場合、途中でAPIの実行上限に引っかかるので注意が必要です。
    # 動画数500本程度であれば余裕で取得できました。
    while True:
        dic_list = dic_list + output['items']
        search_response = youtube.search().list_next(search_response, output)
        if search_response is None:
            break
        output = search_response.execute()

    df = pd.DataFrame(dic_list)
    #各動画毎に一意のvideoIdを取得
    df1 = pd.DataFrame(list(df['id']))['videoId']
    #各動画毎に一意のvideoIdを取得必要な動画情報だけ取得
    df2 = pd.DataFrame(list(df['snippet']))[['channelTitle','publishedAt','channelId','title','description']]
    ddf = pd.concat([df1,df2], axis = 1)

    return ddf

In [ ]:
# 動画全量のID等を取得します
df = get_video_info('snippet', ch_id)
df

In [ ]:
# API の上限に引っかかった場合を考慮し、一度取得できたデータを csv に出力しています
import csv
df.to_csv('出力パス', encoding='utf8', quoting=csv.QUOTE_ALL, index=False)

In [ ]:
# チャンネル概要など、動画とは関係ない情報が取れてしまうので、除外します
df = df.dropna(subset=['videoId']).reset_index()

In [ ]:
def get_statistics(id):
    """
    videoId に紐付いた動画の再生回数や高評価数などのデータを取得します。
    
    id: videoId
    """
    statistics = youtube.videos().list(part = 'statistics', id = id).execute()['items'][0]['statistics']
    return statistics

In [ ]:
# 動画の詳細情報を取得します
df_static = pd.DataFrame(list(df['videoId'].apply(lambda x : get_statistics(x))))
df_output = pd.concat([df,df_static], axis = 1)
df_output

In [ ]:
df_output.to_csv('出力パス', encoding='utf8', quoting=csv.QUOTE_ALL, index=False)